### Genetski algoritam

### Znacajni parametri

In [169]:
TOURNAMENT_SIZE = 10
POPULATION_SIZE = 250
MUTATION_RATE = 0.1
MAX_ITERATION = 8500
ELITE_SIZE = 60

In [119]:
import random

class Individual:
    
    
    def __init__(self,graph):
        self.graph = graph
        self.colors = self.initColors(graph)
        self.fitness = 0
        self.oldValue = 0
        
    def __lt__(self,other):
        return self.fitness < other.fitness
    
    def findAdjColors(self,i):
        adjColors = []
        for ajd in self.graph[i]:
            adjColors.append(self.colors[ajd])
        return adjColors
    
    def isFeasible(self):
        for i in range(len(self.graph)):
            for neigh in self.graph[i]:
                if(self.colors[i]==self.colors[neigh]):
                    return False
            
        return True
        
        # biramo resenje iz okoline
# random odaberemo indeks i smanjimo mu vrednost boje   
    def adjacentSolution(self):
        position = random.randrange(0,len(self.graph))
        self.oldValue = self.colors[position]
      #  if self.colors[position]==1:
       #     self.colors[position]=self.colors[position]+1
        #else:
         #   self.colors[position]=self.colors[position]-1
      
        newColor = random.randrange(1,len(self.graph)+1)
        self.colors[position] = newColor # brze konvergira ako ovako biramo susedno
        if self.isFeasible():
            return position
        self.colors[position] = self.oldValue
        return -1    
        
        
    def correctNonFeasible(self):
        
        for i in range(len(self.graph)):
            adjColors = self.findAdjColors(i)
            if(self.colors[i] in adjColors):
                allColors = list(range(1,len(self.graph) + 1))
                possibleColors = list(set(allColors)-set(adjColors))
                self.colors[i] = random.choice(possibleColors)
        
            
    def fitnessFunction(self,colors):
        value = 0
        for color in colors:
            value+=color
        return value
    
    def restore(self,position):
        self.colors[position] = self.oldValue
    
    def initColors(self,graph):
        colors = []
        numberOfColors = len(graph)
        for i in range(numberOfColors):
            colors.append(random.randrange(1,numberOfColors+1))
        return colors

    def __str__(self):
        print('Bojenje' + str(self.colors))



In [120]:
# Selekcija -> biramo jedinke iz populacije za generisanje naredne
# Koristimo turnirsku: za pocetak 6 turnira 
def tournamentSelection(population):
    min = float('inf')
    k=-1
    for j in range(TOURNAMENT_SIZE):
        j = random.randrange(POPULATION_SIZE) # 100 -> broj populacije
        if population[j].fitness < min:
            min = population[j].fitness
            k=j
    
    return k


In [121]:
def rouletteSelection(population):
    totalFitness = sum([individual.fitness for individual in population])
    randomValue = random.randrange(0,totalFitness)
    currentSum = 0
    
    for i in range(len(population)):
        currentSum += population[i].fitness
        if currentSum > randomValue:
            return i
     

In [122]:
# Jednopoziciono ukrstanje
def crossover(parent1,parent2,child1,child2):
    numberColors = len(parent1.colors)
    # biramo poziciju ukrstanja
    position = random.randrange(numberColors)
    # razmenjujemo
    # prvi deo bojenja
    for i in range(position):
        child1.colors[i]=parent1.colors[i]
        child2.colors[i]=parent2.colors[i]
    
    # drugi deo bojenja
    for j in range(position,numberColors):
        child1.colors[j]=parent2.colors[j]
        child2.colors[j]=parent1.colors[j]
        
    # menjamo da budu dopustiva
    child1.correctNonFeasible()
    child2.correctNonFeasible()
    

In [123]:
# Mutacija
# Verovatnoca za mutaciju 0.05 (parametar, probati druge)
def mutation(individual):
    numberColors = len(individual.colors)
    for i in range(numberColors):
        if random.random() > MUTATION_RATE:
            continue
        individual.colors[i] = random.choice(individual.colors)
        individual.correctNonFeasible()
        

In [124]:
# Inicijalna populacija
# 100 jedinki u populaciji (parametar, probati druge)
# za
def initPopulation(graph):
    population = []
    newPopulation = []
    for i in range(POPULATION_SIZE):
        individual = Individual(graph)
        individual.initColors(graph)
        individual.correctNonFeasible()
        individual.fitness = individual.fitnessFunction(individual.colors)
        population.append(individual)
        newPopulation.append(individual)
        
    return (population,newPopulation)



In [125]:
 def simulatedAnnealing(individual, maxIters):
    for i in range(maxIters):
        j = individual.adjacentSolution()
        if j < 0:
            continue
        newFitness = individual.fitnessFunction(individual.colors)
        if newFitness < individual.fitness:
            individual.fitness = newFitness
        else:
            p = 1.0 / (i + 1) ** 0.5
            q = random.uniform(0, 1)
            if p > q:
                individual.fitness = newFitness
            else:
                individual.restore(j)

In [154]:
def ga(graph):
    # Dobijamo inicijalnu populaciju
    population,newPopulation = initPopulation(graph)
    
    for iteration in range(MAX_ITERATION):
        population.sort() # da bi nam prvi bio sa najboljim fitnesom tj minimalnom sumom
        # Elitizam, cuvamo 30 najboljih
        for i in range(ELITE_SIZE):
            newPopulation[i] = population[i]
        # Ostale zamenjujemo decom
        for i in range(ELITE_SIZE,POPULATION_SIZE,2):
            ind1 = rouletteSelection(population)
            ind2 = rouletteSelection(population)
            crossover(population[ind1],population[ind2],newPopulation[i],newPopulation[i+1])
            mutation(newPopulation[i])
            mutation(newPopulation[i+1])
            newPopulation[i].fitness = newPopulation[i].fitnessFunction(newPopulation[i].colors)
            newPopulation[i+1].fitness = newPopulation[i+1].fitnessFunction(newPopulation[i+1].colors)
    
        population=newPopulation

    population.sort()

    print('Resenje',population[0].fitness)
    return population[0].fitness
    # Sa ruletskom se dobija 12


#### Genetski + simulirano kaljenje

In [130]:

def hybrid_ga(graph):
    # Dobijamo inicijalnu populaciju
    population,newPopulation = initPopulation(graph)

    for iteration in range(MAX_ITERATION):
        population.sort() # da bi nam prvi bio sa najboljim fitnesom tj minimalnom sumom
        # Elitizam, cuvamo 30 najboljih
        for i in range(ELITE_SIZE):
            newPopulation[i] = population[i]
        # Ostale zamenjujemo decom
        for i in range(ELITE_SIZE,POPULATION_SIZE,2):
            ind1 = tournamentSelection(population)
            ind2 = tournamentSelection(population)
            crossover(population[ind1],population[ind2],newPopulation[i],newPopulation[i+1])
            mutation(newPopulation[i])
            mutation(newPopulation[i+1])
            newPopulation[i].fitness = newPopulation[i].fitnessFunction(newPopulation[i].colors)
            newPopulation[i+1].fitness = newPopulation[i+1].fitnessFunction(newPopulation[i+1].colors)
    # sprovodimo kaljenje na najbolji u populaciji
        simulatedAnnealing(newPopulation[0], 10)
        population=newPopulation

    population.sort()

    print('Resenje',population[0].fitness)
    return population[0].fitness
    # Sa ruletskom se dobija 12
        

#### Primena genetskog i hibridnog na grafove

In [11]:
import time

In [141]:
graph0= [[3],[3],[3],[0,1,2,4],[3,5,6,7],[4],[4],[4]]
start = time.perf_counter()
solution = ga(graph0)
solutionHybrid = hybrid_ga(graph0)
end = time.perf_counter()
print(end-start)
timeExec = end - start

string = "\nIme grafa: graph0" + '\n' + "Resenje "+str(solution)+" Resenje hybrid: "+str(solutionHybrid)+"\nParametri\n" + "Velicina turnira " + str(TOURNAMENT_SIZE) + "\nVelicina populacije " + str(POPULATION_SIZE) + "\n"+ "Verovatnoca mutacije " + str(MUTATION_RATE) + "\n"+ "Maksimalno iteracija " + str(MAX_ITERATION) + "\n"+ "Velicina elitizma " + str(ELITE_SIZE) + "\n" + "Vreme izvrsavanja " + str(timeExec) + "\n"  + "******************************************************" + "\n"

file = open("Parametri_genetski.txt","a")
file.write(string)



Resenje 11
Resenje 11
4.304967870004475


265

In [139]:
graph1 = [[1, 3, 6, 8], [0, 2, 5, 7], [1, 4, 6, 9], 
          [0, 4, 5, 9], [2, 3, 7, 8], 
          [1, 3, 10], [0, 2, 10],
          [1, 4, 10], [0, 4, 10], [2, 3, 10], 
          [5, 6, 7, 8, 9]]
start = time.perf_counter()
solution = ga(graph1)
solutionHybrid = hybrid_ga(graph1)
end = time.perf_counter()
print(end-start)
timeExec = end - start

string = "\nIme grafa: graph1" + '\n' + "Resenje "+str(solution)+" Resenje hybrid " +str(solutionHybrid)+"\nParametri\n" + "Velicina turnira " + str(TOURNAMENT_SIZE) + "\nVelicina populacije " + str(POPULATION_SIZE) + "\n"+ "Verovatnoca mutacije " + str(MUTATION_RATE) + "\n"+ "Maksimalno iteracija " + str(MAX_ITERATION) + "\n"+ "Velicina elitizma " + str(ELITE_SIZE) + "\n" + "Vreme izvrsavanja " + str(timeExec) + "\n"  + "******************************************************" + "\n"

file = open("Parametri_genetski.txt","a")
file.write(string)
file.close()

Resenje 21
Resenje 21
5.794084228997235


#### Veliki graf

In [155]:
graph2 =[[1, 3, 6, 8, 12, 14, 17, 19, 24, 26, 29, 31, 35, 37, 40, 42],
 [0, 2, 5, 7, 11, 13, 16, 18, 23, 25, 28, 30, 34, 36, 39, 41],
 [1, 4, 6, 9, 12, 15, 17, 20, 24, 27, 29, 32, 35, 38, 40, 43],
 [0, 4, 5, 9, 11, 15, 16, 20, 23, 27, 28, 32, 34, 38, 39, 43],
 [2, 3, 7, 8, 13, 14, 18, 19, 25, 26, 30, 31, 36, 37, 41, 42],
 [1, 3, 10, 12, 14, 21, 24, 26, 33, 35, 37, 44],
 [0, 2, 10, 11, 13, 21, 23, 25, 33, 34, 36, 44],
 [1, 4, 10, 12, 15, 21, 24, 27, 33, 35, 38, 44],
 [0, 4, 10, 11, 15, 21, 23, 27, 33, 34, 38, 44],
 [2, 3, 10, 13, 14, 21, 25, 26, 33, 36, 37, 44],
 [5, 6, 7, 8, 9, 16, 17, 18, 19, 20, 28, 29, 30, 31, 32, 39, 40, 41, 42, 43],
 [1, 3, 6, 8, 22, 24, 26, 29, 31, 45],
 [0, 2, 5, 7, 22, 23, 25, 28, 30, 45],
 [1, 4, 6, 9, 22, 24, 27, 29, 32, 45],
 [0, 4, 5, 9, 22, 23, 27, 28, 32, 45],
 [2, 3, 7, 8, 22, 25, 26, 30, 31, 45],
 [1, 3, 10, 22, 24, 26, 33, 45],
 [0, 2, 10, 22, 23, 25, 33, 45],
 [1, 4, 10, 22, 24, 27, 33, 45],
 [0, 4, 10, 22, 23, 27, 33, 45],
 [2, 3, 10, 22, 25, 26, 33, 45],
 [5, 6, 7, 8, 9, 22, 28, 29, 30, 31, 32, 45],
 [11,12,13,14,15,16,17,18,19,20,21,34,35,36,37,38,39,40,41,42,43,44],
 [1, 3, 6, 8, 12, 14, 17, 19, 46],
 [0, 2, 5, 7, 11, 13, 16, 18, 46],
 [1, 4, 6, 9, 12, 15, 17, 20, 46],
 [0, 4, 5, 9, 11, 15, 16, 20, 46],
 [2, 3, 7, 8, 13, 14, 18, 19, 46],
 [1, 3, 10, 12, 14, 21, 46],
 [0, 2, 10, 11, 13, 21, 46],
 [1, 4, 10, 12, 15, 21, 46],
 [0, 4, 10, 11, 15, 21, 46],
 [2, 3, 10, 13, 14, 21, 46],
 [5, 6, 7, 8, 9, 16, 17, 18, 19, 20, 46],
 [1, 3, 6, 8, 22, 46],
 [0, 2, 5, 7, 22, 46],
 [1, 4, 6, 9, 22, 46],
 [0, 4, 5, 9, 22, 46],
 [2, 3, 7, 8, 22, 46],
 [1, 3, 10, 22, 46],
 [0, 2, 10, 22, 46],
 [1, 4, 10, 22, 46],
 [0, 4, 10, 22, 46],
 [2, 3, 10, 22, 46],
 [5, 6, 7, 8, 9, 22, 46],
 [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 46],
 [23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45]]

In [170]:
start = time.perf_counter()
solution = ga(graph2)
end = time.perf_counter()
print(end-start)
timeExecGA = end - start
start = time.perf_counter()
solutionHybrid = hybrid_ga(graph2)
end = time.perf_counter()
timeHybExec = end - start
string = "\nIme grafa: graph2" + '\n' + "Resenje "+str(solution)+" Resenje hybrid " +str(solutionHybrid)+"\nParametri\n" + "Velicina turnira " + str(TOURNAMENT_SIZE) + "\nVelicina populacije " + str(POPULATION_SIZE) + "\n"+ "Verovatnoca mutacije " + str(MUTATION_RATE) + "\n"+ "Maksimalno iteracija " + str(MAX_ITERATION) + "\n"+ "Velicina elitizma " + str(ELITE_SIZE) + "\n" + "Vreme izvrsavanja: ga " + str(timeExecGA) +"hybrid "+str(timeHybExec)+"\n"  + "******************************************************" + "\n"

file = open("Parametri_genetski.txt","a")
file.write(string)
file.close()

Resenje 181
903.1404078640044
Resenje 96
